# IMPORT

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [ ]:
from scipy.spatial.distance import pdist, squareform
def VAT(R):
    R = np.array(R)
    N, M = R.shape
    if N != M:
        R = squareform(pdist(R))
    J = list(range(0, N))
    
    y = np.max(R, axis=0)
    i = np.argmax(R, axis=0)
    j = np.argmax(y)
    y = np.max(y)
    
    I = i[j]
    del J[I]
    
    y = np.min(R[I,J], axis=0)
    j = np.argmin(R[I,J], axis=0)
    I = [I, J[j]]
    J = [e for e in J if e != J[j]]
    C = [1,1]

    for r in range(2, N-1):
        y = np.min(R[I,:][:,J], axis=0)
        i = np.argmin(R[I,:][:,J], axis=0)
        j = np.argmin(y)
        y = np.min(y)
        I.extend([J[j]])
        J = [e for e in J if e != J[j]]
        C.extend([i[j]])

    y = np.min(R[I,:][:,J], axis=0)
    i = np.argmin(R[I,:][:,J], axis=0)

    I.extend(J)
    C.extend(i)

    RI = list(range(N))
    for idx, val in enumerate(I):
        RI[val] = idx

    RV = R[I,:][:,I]
    return RV.tolist(), C, I


def entropy(probs):
    
    return -probs.dot(np.log2(probs))


def mutual_info(df):
    
    Hx = entropy(df.iloc[:,0].value_counts(normalize=True, sort=False))
    Hy = entropy(df.iloc[:,1].value_counts(normalize=True, sort=False))
    
    counts = df.groupby(list(df.columns.values)).size()
    probs = counts/ counts.values.sum()
    H_xy = entropy(probs)

    # Mutual Information
    I_xy = Hx + Hy - H_xy
    MI = I_xy
    NMI = I_xy/min(Hx,Hy) #I_xy/np.sqrt(H_x*H_y)
    return MI
    return {'H_'+list(df)[0]:Hx,'H_'+list(df)[1]:Hy,'MI':MI,'NMI':NMI} 


In [ ]:
food = pd.read_csv("food_nutrient_2011_13_AHS.csv", header=0,low_memory=False)

# QUESTION 1

In [ ]:
##Question 1


###1a

foodcut = food.loc[:, "Energy, with dietary fibre (kJ)":"Total trans fatty acids (mg)"]
foodcut.columns = food.loc[:, "Energy, with dietary fibre (kJ)":"Total trans fatty acids (mg)"].columns

###1b

foodcut = foodcut.astype(float)
scaler = StandardScaler().fit(foodcut)
foodscaled = scaler.transform(foodcut)

###1c

print("***")
print("Number of rows: ", foodscaled.shape[0])
print("Number of columns: ", foodscaled.shape[1])
print("Min: ", round(foodscaled.min(), 1))
print("Max: ", round(foodscaled.max(), 1))
print("Mean: ", round(foodscaled.mean(), 1))
print("Standard Deviation: ", round(foodscaled.std(), 1))


# QUESTION 2

In [ ]:
#Question 2


###2a

EnergyLevel = [1 if x is x>1000 else 0 for x in food["Energy, with dietary fibre (kJ)"]]

###2b

sklearn_pca = PCA(n_components=2)   #we want just the first two PCs
foodreduced = sklearn_pca.fit_transform(foodscaled)

###2c

color = ["red" if x is x==1 else "blue" for x in EnergyLevel]

plt.rcParams['figure.figsize'] = [15, 10]    # change plot size
plt.scatter(foodreduced[:,0],foodreduced[:,1],  marker="x", c=color)   #plot the PC's
plt.scatter([],[], marker="x",color='r')
plt.xlabel('1st Principal Component', fontsize=15)
plt.ylabel('2nd Principal Component', fontsize=15)
plt.legend(["Low Energy", "High Energy"])
plt.title("PCA on Food (2 Components)", fontsize=20)
plt.show()


## Question 2d

The scatter plot shows the standardised food data reduced down to the 2 dimensions with the most variation. The types of food with low energy are clustered together, similiar to the high energy food. Principal Component 1 is most likely to be the overall amount of energy in a type of food as mostly all the high energy (red marks) foods are to the right of the low energy (blue marks) food. Principal component 2 is most likely all the vitamins and nutrients as it has a high variability and generally lower energy food has more of it. The advantage of using PCA on this dataset is that it is much easier to visualise due to the small number of features. The disadvantage is that most of the variability is lost as only about 0.25 of the variance is captured with a PCA of 2 components on this dataset. 

# QUESTION 3

In [ ]:
#Question 3


###3a

food["Survey ID"] = food["Survey ID"].astype("str")
category = [x[:2] for x in food["Survey ID"]]    # slice the first two characters
food["Food Category"] = category

###3b

foodscaledsample = pd.DataFrame(foodscaled)    # convert into a dataframe
foodscaledsample["Food Category"] = category    # add the Food Category column
foodscaledsample = foodscaledsample.loc[foodscaledsample["Food Category"].isin(["13","20","24"])]    # select foods
foodscaledsample = foodscaledsample.drop(columns=["Food Category"])

###3c

RV, C, I = VAT(foodscaledsample)

###3d

plt.axis('equal')
x=sns.heatmap(RV,cmap='Spectral',xticklabels=False,yticklabels=False)
x.set(xlabel="Features", ylabel="Features")
plt.title("Heat Map of Dissimilarity Matrix", fontsize=15)
plt.show()


## Question 3e

2 clusters can be identified from the heatmap. This is not expected as 3 food categories were selected from the foodscaled data and so 3 clusters were expected. The use of different colormaps produce visualisations of varying usefulness as some colormaps make it hard to distinguish the data because of the colour. A Colormap that is sequential is good for this dataset as the data has ordering. The colormap also should change colour over a small change in data value as this data set has is very evenly distributed and small differences need to be identified.


# QUESTION 4

In [ ]:
#Question 4

###4a 

foodscaled = pd.DataFrame(foodscaled)
foodscaled.columns = foodcut.columns

sse = {}
for k in range(2, 25):
    kmeans = KMeans(n_clusters=k, random_state=100).fit(foodscaled)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()), "r-x")
plt.xlabel("Number of clusters", fontsize=15)
plt.ylabel("SSE", fontsize=15)
plt.title("Number of Clusters vs SSE", fontsize=20)
plt.show()


## Question 4b

As the number of clusters increases, the SSE decreases monotonically. The shape and trend of the graph looks linear and is smooth with an elbow being very hard to spot. It is not obvious where the elbow is but if one must be chosen, it would be at around 16 clusters. This is to be expected as from previous visualisations such as the heatmap, it is evident that the data is very evenly distributed and obvious clusters would be hard to identify.

# QUESTION 5

In [ ]:
###Question 5


###5a

foodcorr = food.loc[:, "Energy, with dietary fibre (kJ)":"Added sugars (g)"].corr(method='pearson')
x=sns.heatmap(foodcorr,cmap='RdYlBu',xticklabels=True,yticklabels=True)
plt.title("Pearson correlation matrix for the first 10 nutrients", fontsize=20)
plt.show()


In [ ]:
###5b code

mi = {}

x = pd.cut(food["Protein (g)"], 2, labels=False)
y = pd.cut(food["Energy, with dietary fibre (kJ)"], 2, labels=False)
temp = pd.DataFrame({'0': x,'1': y})
mi[2] = mutual_info(temp)
    
for i in range(10,200,10):
    x = pd.cut(food["Protein (g)"], i, labels=False)
    y = pd.cut(food["Energy, with dietary fibre (kJ)"], i, labels=False)
    temp = pd.DataFrame({'0': x,'1': y})
    mi[i] = mutual_info(temp)

plt.plot(list(mi.keys()), list(mi.values()), "r-x")
plt.xlabel("Number of equal-width bins", fontsize=15)
plt.ylabel("Mutual Information", fontsize=15)
plt.title("Number of equal-width bins vs Mutual Information, for Protein (g) and Energy, with dietary fibre (kJ)", fontsize=20)
plt.show()


## Question 5b

As the number of equal-width bins increases, the mutual information for protein and energy increases greatly from 2 to 30 bins and then steadily from 30 bins onwards.

In [ ]:
###5c code

def get_redundant_pairs(df):
    # Get the diagonal and lower triangular pairs of the correlation matrix
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations_pearson(df, n):
    au_corr = df.corr().unstack()    # remove unwanted mutual information of certain pairs
    labels_to_drop = get_redundant_pairs(df)
    # remove unwanted mutual information of certain pairs and sort by highest to lowest
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]    # return the top n

    

def get_top_abs_correlations_mi(df, n):
    # calculate MI dictionary
    mi_corr_dict = {}
    for i in range(df.shape[1]):
        mi_corr_dict[df.columns[i]] = []
        for j in range(df.shape[1]):
            x = pd.cut(df[df.columns[i]], 20, labels=False)
            y = pd.cut(df[df.columns[j]], 20, labels=False)
            temp = pd.DataFrame({'0': x,'1': y})
            mi_corr_dict[df.columns[i]].append(mutual_info(temp))
    # turn the MI dictionary into a matrix
    mi_corr_df = pd.DataFrame(mi_corr_dict)
    
    # rename all the rows to be the same as the columns
    cols = mi_corr_df.columns.tolist()
    mapper = {}
    j = 0
    for i in cols:
        mapper[j] = i
        j += 1
    mi_corr_df = mi_corr_df.rename(mapper, axis="index")
    
    au_corr = mi_corr_df.unstack()    # turn the dataframe into a series
    labels_to_drop = get_redundant_pairs(df)
    # remove unwanted mutual information of certain pairs and sort by highest to lowest
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]    # return the top n

print("Top 10 Pearson feature pairs")
print(get_top_abs_correlations_pearson(foodcut, 10))
print("***")
print("Top 10 Mutual Information feature pairs")
print(get_top_abs_correlations_mi(foodcut, 10))

## Question 5c

The top Pearson pairs and the top Mutual Information pairs do share some feature pairs such as, Energy with dietary fibre & Energy without dietary fibre. The difference between the top Pearson and the top Mutual Information is that the Pearson can only find linear relationships between features while Mutual Information can find non-linear relationships as well. 

# QUESTION 6

In [ ]:
##Question 6


###6a

X_train, X_test, y_train, y_test = train_test_split(foodscaled, category, train_size=0.80, test_size=0.20)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print("***")
print("Q6.a: Train Test Split Results")
print("X_train matrix: ", X_train.shape)
print("y_train labels: ", len(y_train))
print("X_test matrix: ", X_test.shape)
print("y_test labels: ", len(y_test))
print("***")

###6b

maxdepth = {}
for k in range(1,41):
    dt = DecisionTreeClassifier(criterion="entropy", max_depth=k)
    dt.fit(X_train, y_train)
    y_pred=dt.predict(X_test)
    maxdepth[k] = accuracy_score(y_test, y_pred)

plt.plot(list(maxdepth.keys()), list(maxdepth.values()), "r-x")
plt.xlabel("Depth of Decision Tree", fontsize=15)
plt.ylabel("Accuracy", fontsize=15)
plt.title("Depth of Decision Tree vs Accuracy", fontsize=20)
plt.show()


## Question 6c

The accuracy of the decision tree increases dramatically from depth 1 to depth 8 and starts to peak at depth 10. After a depth of 10, the accuracy does not increase anymore. A reason for the rapid increase from depth 1 to 8 is because the decision tree's depth is limited to a small number and not all the data points can be classified by just a few features. The accuracy then plateaus after a certain depth (in this case depth 10) as no more additonal information can be gained about the data point. The decision tree is also very accurate as the training data is normalised with the testing data.

# QUESTION 7

In [ ]:
#Question 7

 ###7a

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

y_pred_test = knn.predict(X_test)
y_pred_train = knn.predict(X_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
train_accuracy = accuracy_score(y_train, y_pred_train)

print("***")
print("Q7a: Food category prediction using k-NN (k=1)")
print("Train accuracy: ", round(train_accuracy*100, 1),"%")
print("Test accuracy: ", round(test_accuracy*100, 1),"%")
print("***")

##7b

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred_test = knn.predict(X_test)
y_pred_train = knn.predict(X_train)

test_accuracy = accuracy_score(y_test, y_pred_test)
train_accuracy = accuracy_score(y_train, y_pred_train)

print("***")
print("Q7b: Food category prediction using k-NN (k=3)")
print("Train accuracy: ", round(train_accuracy*100, 1),"%")
print("Test accuracy: ", round(test_accuracy*100, 1),"%")
print("***")


## Question 7c

Testing the training data with k=1 will always yield 100% accuracy but with k=3 it will not be 100% accurate as there is added noise in the prediction. The test data accuracy for k=1 is generally higher than when k=3 also possibly because of the added noise from the extra comparisons with the other 2 data points.

## Question 7d

The accuracy of the KNN predictions are over optimistic because the testing data is being standardised with the training data with a mean of 0 and std of 1. The predictions are therefore more accurate because testing data is not independent of the training data. To fix this, the scaler (normalisation) should be applied to the training data only. Then the same scalar is applied onto the testing data.


In [ ]:
#Question 8 code

# feature generation
foodscaled["Total fat (g) x Protein (g)"] = foodscaled["Total fat (g)"]*foodscaled["Protein (g)"]

# clustering labels
kmeans = KMeans(n_clusters=16, random_state=100).fit(foodscaled)
foodscaled["clusterlabel"] = kmeans.labels_

# Question 8 discussion

feature generation is unlikely to increase accuracy as it is derived data from the data already there. It only adds noise to the data and reduces accuracy. Clustering labels can increase accuracy as it removes the features that do not have any relation to the label and thus keep all the "important" features.